<a href="https://colab.research.google.com/github/shalabhs45/ML/blob/main/BaggingandBoosting/Bagging%26Boosting_Assignment_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with a house price prediction dataset and see how the different techniques we learned in this module like decision trees, bagged trees and boosted trees perform on this problem.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
house_price_data = pd.read_csv('https://raw.githubusercontent.com/shalabhs45/ML/main/BaggingandBoosting/data/train.csv')
house_price_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## Dataset

In this exercise you'll work with a House Price Prediction Dataset. You will predict the house price based on house features like location, lot area, front area, street access etc.

Create a directory called datasets in your Google Drive and import the files train.csv in that directory. You can check more avout the data from data_description.txt file.

Load the data in pandas dataframe and inspect the shape and few rows of the data

In [6]:
#house_price_data = pd.read_csv('/gdrive/MyDrive/datasets/train.csv')
print(house_price_data.shape)
house_price_data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
house_price_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


### Create a train and test split (80% train - 20% test)

In [8]:
from sklearn.model_selection import train_test_split
X = house_price_data.drop(['Id','SalePrice'],axis=1)
y = house_price_data['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Handle the NaN values
Observe multiple columns have a lot of NaN values. Let impute those missing values with zero. You can use more complex imputation techniques if you deem necessary.

### Process the string features using [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

There are multiple string/non-number fetures in the data. We use label encoding to convert them to numbers.

In [ ]:
from sklearn.preprocessing import LabelEncoder


## Import necessary libraries

In [ ]:
#Import necessary metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#Import necessary model selection tools
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

#Import necessary models
#Simple decision tree
from sklearn.tree import DecisionTreeRegressor
# Simple bagged model
from sklearn.ensemble import BaggingRegressor
# Simple boosted model
from xgboost import XGBRegressor

## Train a Simple Decision Tree Rgeression Model

We will use DecisionTreeRegressor from scikit learn libarry. Explore the [API](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) and model parameters before diving in.

### GridSearch some of the parameters like max_depth and max_features to find the best possible decison tree. We use GridSearchCV API for doing this.

In [ ]:
# You can use this sample code or write your own code
# model = DecisionTreeRegressor(random_state=1)
# dt_parameters = {
# 'max_depth':[2,4,8,12,16,24,32],
# 'max_features':['auto','sqrt','log2']}
# dt_search = GridSearchCV(model,dt_parameters,cv=5,scoring='r2')
# dt_search.fit(X_train, y_train)

Print the following
1. best parameters
2. R2 score of the best model.
3. Mean squared error of the best model

In [ ]:
# You can use this sample code or write your own code
# best_model = dt_search.best_estimator_
# best_params = dt_search.best_params_
# prediction = best_model.predict(X_test)
# print("best parameters:",best_params)
# print("r2 score of best decision tree model: ",r2_score(prediction,y_test))
# print("Mean squared error of best decision tree model: ",mean_squared_error(prediction,y_test))

### Fit multiple decision tree regressors on the house price data and plot the variance in accuracy of a simple decision tree. Use the best set of parameters found in grid search for this model.

In [ ]:
# You can use this sample code or write your own code
# # evaluate the model
# cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)
# n_scores = cross_val_score(best_model, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
# # report performance
# print('Avg. error: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
# sns.distplot(n_scores)

## Train a Bagged Decison Tree Model

We will use BaggingRegressor from scikit learn libarry. Explore the [API](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) and model parameters before diving in.

### GridSearch some of the parameters to find the best possible bagged regressor.

Print the following
1. best parameters
2. R2 score of the best model.
3. Mean squared error of the best model

### Fit multiple bagging regressors on the house price data and plot the variance in accuracy of a simple decision tree. Use the best set of parameters found in grid search for this model.

 ### Compare the r2 score distribution graph with what we observed in decison trees.

Print the mean and standard deviation of decision tree and bagging model cv scores

## Train Gradient Boosted Decision Tree

We will use XGBRegressor from XGBoost library Explore the [API](https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn) and model parameters before diving in. In parctice this is the model you will use most practical use cases.

### GridSearch some of the parameters to find the best possible XGBoost Regressor.

Print the following
1. best parameters
2. R2 score of the best model.
3. Mean squared error of the best model

### Fit multiple bagging regressors on the house price data and plot the variance in accuracy of a simple decision tree. Use the best set of parameters found in grid search for this model.

 ### Compare the r2 score distribution graph with what we observed in bagged regressor. Note your observations.

Print the mean and standard deviation of xgboost and bagging model cv scores